# Continue Restructuring Labels 

Continuation of `2020-03-15-RestructureLabels.ipynb`

```
gsv_panorama_id,sv_image_x,sv_image_y,label_type_id,photagrapher_heading,heading,is_researcher

stxXyCKAbd73DmkM2vsIHA,9676,-455,1,50.4137916565,253.375,t
```

In [2]:
import pandas as pd
import os
import s3fs # for reading from S3FileSystem
import json # for working with JSON files 

import matplotlib.pyplot as plt

pd.set_option('max_colwidth', -1)

# Load Needed CSVs
* restructure_single_labels.csv - in local directory
* EDA\Formatting_Data\data\meta_with_depth.csv - in EDA folder, needed to get the gsv_panorama_id, heading, etc

In [7]:
df_labels = pd.read_csv('restructure_single_labels.csv')
print(df_labels.columns)
df_labels.head()

Index(['filename', 'file_size', 'file_attributes', 'region_count', 'region_id',
       'region_shape_attributes', 'region_attributes', 'img_id',
       'present_ramp', 'missing_ramp', 'obstacle', 'surface_prob',
       'no_sidewalk', 'null', 'sv_image_x', 'sv_image_y',
       'sv_image_y_bottom_origin', 'label_name'],
      dtype='object')


,filename,file_size,file_attributes,region_count,region_id,region_shape_attributes,region_attributes,img_id,present_ramp,missing_ramp,obstacle,surface_prob,no_sidewalk,null,sv_image_x,sv_image_y,sv_image_y_bottom_origin,label_name
0,680_45.jpg,49558,{},3,0,"{""name"":""point"",""cx"":108,""cy"":389}","{""Present Curb Ramp"":""1\n""}",680,True,False,False,False,False,False,108,389,251,Present Curb Ramp
1,680_45.jpg,49558,{},3,1,"{""name"":""point"",""cx"":160,""cy"":389}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False,160,389,251,Present Curb Ramp
2,680_45.jpg,49558,{},3,2,"{""name"":""point"",""cx"":607,""cy"":453}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False,607,453,187,Present Curb Ramp
3,680_135.jpg,51194,{},6,0,"{""name"":""point"",""cx"":18,""cy"":475}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False,18,475,165,Present Curb Ramp
4,680_135.jpg,51194,{},6,1,"{""name"":""point"",""cx"":245,""cy"":413}","{""Present Curb Ramp"":""1""}",680,True,False,False,False,False,False,245,413,227,Present Curb Ramp


In [8]:
SAGEMAKER_PATH = r'/home/ec2-user/SageMaker'
DEPTH_LOC = r'EDA/Formatting_Data/data/meta_with_depth.csv'

df_meta = pd.read_csv(os.path.join(SAGEMAKER_PATH, DEPTH_LOC))
print(df_meta.columns)
df_meta.head()

Index(['Unnamed: 0', 'date', 'lat', 'long', 'pano_id', 'name', 'pano_yaw_deg',
       'tilt_yaw_deg', 'tilt_pitch_deg'],
      dtype='object')


,Unnamed: 0,date,lat,long,pano_id,name,pano_yaw_deg,tilt_yaw_deg,tilt_pitch_deg
0,0,2019-06,42.957503,-87.938367,XPRpjNDhowVo8zvqvSU1CA,1,91.979996,125.040000,0.83
1,1,2016-10,42.899259,-88.047098,iEyn0apLSZvl4i4alUbfcA,10,145.819990,-178.269990,1.06
2,2,2011-08,42.921614,-87.881025,1BzC3WoFeJ8U1aUT9Hx8mg,100,359.180000,30.769999,2.55
3,3,2018-09,43.050123,-88.040263,oRN5vilebPS0srDXRPylzw,1000,161.110000,67.549995,1.57
4,4,2019-05,42.959289,-88.026043,KimNSirhP1TzngZkSpc8UA,10000,270.060000,-114.810000,1.35


In [12]:
labels_keep = ['filename', 'file_size', 'region_count', 'region_id', 'img_id',
       'present_ramp', 'missing_ramp', 'obstacle', 'surface_prob',
       'no_sidewalk', 'sv_image_x', 'sv_image_y','sv_image_y_bottom_origin', 'label_name']
meta_keep = ['date', 'lat', 'long', 'pano_id', 'name', 'pano_yaw_deg',
       'tilt_yaw_deg', 'tilt_pitch_deg']

df_combine = pd.merge(df_labels[labels_keep], df_meta[meta_keep], how = 'left', left_on = 'img_id', right_on = 'name')
print(df_combine.shape)
df_combine.head()

(2851, 22)


,filename,file_size,region_count,region_id,img_id,present_ramp,missing_ramp,obstacle,surface_prob,no_sidewalk,...,sv_image_y_bottom_origin,label_name,date,lat,long,pano_id,name,pano_yaw_deg,tilt_yaw_deg,tilt_pitch_deg
0,680_45.jpg,49558,3,0,680,True,False,False,False,False,...,251,Present Curb Ramp,2017-08,43.036286,-87.907481,a5Q6wg2P8gqUtZemvQJU9Q,680,239.89,-160.06,1.75
1,680_45.jpg,49558,3,1,680,True,False,False,False,False,...,251,Present Curb Ramp,2017-08,43.036286,-87.907481,a5Q6wg2P8gqUtZemvQJU9Q,680,239.89,-160.06,1.75
2,680_45.jpg,49558,3,2,680,True,False,False,False,False,...,187,Present Curb Ramp,2017-08,43.036286,-87.907481,a5Q6wg2P8gqUtZemvQJU9Q,680,239.89,-160.06,1.75
3,680_135.jpg,51194,6,0,680,True,False,False,False,False,...,165,Present Curb Ramp,2017-08,43.036286,-87.907481,a5Q6wg2P8gqUtZemvQJU9Q,680,239.89,-160.06,1.75
4,680_135.jpg,51194,6,1,680,True,False,False,False,False,...,227,Present Curb Ramp,2017-08,43.036286,-87.907481,a5Q6wg2P8gqUtZemvQJU9Q,680,239.89,-160.06,1.75


In [11]:
df_combine['pano_id'].describe()

count     2851                  
unique    506                   
top       hYQQHPPHC13-fYlYnzVIWw
freq      23                    
Name: pano_id, dtype: object

In [13]:
df_combine.to_csv('single_labels_with_metadata.csv', index = False)

In [14]:
df_combine['sv_image_y_bottom_origin'].describe()

count    2851.000000
mean     229.916871 
std      47.643785  
min      6.000000   
25%      210.000000 
50%      236.000000 
75%      263.000000 
max      315.000000 
Name: sv_image_y_bottom_origin, dtype: float64

# Create only Galen's CSV
* gsv_panorama_id = pano_id
* sv_image_x = sv_image_x


In [ ]:
#gsv_panorama_id,sv_image_x,sv_image_y,label_type_id,photagrapher_heading,heading,is_researcher
